In [370]:
# pip install TextBlob

In [371]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from sklearn.cluster import DBSCAN
import collections
import gender_guesser.detector as gender
from textblob import TextBlob

from category_encoders import TargetEncoder

from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', None)

# Basic Understanding 

In [372]:
odf = pd.read_csv('C:\\Users\\xzhao\\Data\\AB_NYC_2019.csv')
df = pd.read_csv('C:\\Users\\xzhao\\Data\\AB_NYC_2019.csv')
# df.head()
print(df.shape)

(48895, 16)


In [373]:
# df.corr()

In [374]:
# df.info()

In [375]:
'''
mannual adjustment
remove 0 dollar listing, they are rare
switch to '' on Nan in name
over write zero reviews in 2019 as reviewed in 2019YE
over wirte brand new listers' relevant reviews as 0, and change data type
'''

df =df[df['price']!=0]
df['name'] = df['name'].fillna('')
df['last_review'] = df['last_review'].fillna('2019-12-31')
df['reviews_per_month'] = df['reviews_per_month'].fillna('0')
df['reviews_per_month']= df['reviews_per_month'].astype(int)

In [376]:
'''
understand feature
unique count and if less than 20, unique value
Nan samples
data description
'''
def ud(feature):
    print('unique value count', df[feature].nunique())
    if df[feature].nunique()<20:
        print('unique value', df[feature].unique())
    print('empty records:',  df[feature].isna().sum(), '\n',df[df[feature].isna()])
    print('data description:', df[feature].describe())
    
    

In [377]:
# ud('name')

In [378]:
# ud('host_id')

In [379]:
# ud('host_name')

In [380]:
# ud('neighbourhood_group')

In [381]:
# ud('neighbourhood')

In [382]:
# ud('room_type')

In [383]:
# ud('price')
# df[df['price'] >150]
# df[df['number_of_reviews'] > 5]

In [384]:
# ud('minimum_nights')
# df.groupby('minimum_nights').size()
# df[df['minimum_nights'] > 800]

In [385]:
# ud('number_of_reviews')

In [386]:
# ud('last_review')

In [387]:
# ud('reviews_per_month')

In [388]:
#no last review  means review per month =0, basically brand new listers
# df[df['last_review'].isna() & df['reviews_per_month'].isna()]

In [408]:
#this is total listings managed accross all airbnb under their name (self manaage + co host)
# ud('calculated_host_listings_count')
# df[df['calculated_host_listings_count'] >300]

In [390]:
# ud('availability_365')
# df[df['availability_365'] ==0]

In [391]:
df.shape

(48884, 16)

In [392]:
# df.groupby()

TypeError: You have to supply one of 'by' and 'level'

# Feature engineering

In [393]:
#pain feature
def paint(feature):
    sns.scatterplot(x=feature, y='price',data=df,alpha = 0.2)

In [409]:
# paint('neighbourhood_group')

In [ ]:
# odf.columns

In [ ]:
'''
3 approach to feature:drop, engineer+encode, simple encode
'id', 
'name', extrac info unavailabe from other columns then one hot encode
'host_id', 
'host_name', guess host gender
'neighbourhood_group', straight one hot
'neighbourhood', straight one hot
'latitude', 'longitude', figure out how to use later
'room_type', one hot
'price', target
'minimum_nights', label encode, >50
'number_of_reviews', label encode, <50
'last_review', break down to lr_year, lr_month, lr_day_of_week then onehot
'reviews_per_month', label <3
'calculated_host_listings_count', label <20
'availability_365', sort value take top 5 as wide variaty

'''

In [ ]:
'''
convert name column to a list of words appeared
screen out nolist then out put top N frequent word
'''
allwords = list(df['name'].str.cat(sep=' ').split())
nolist = ['in','to','the','with','and','of','room','near','&','-','east']
words = []
for word in allwords:
    if word.lower() not in nolist:
        words.append(word.lower())
        
def topNwords(word_list,N):
    # Count the frequency of each word
    word_freq = collections.Counter(word_list)

    # Get the top N most common words
    top_words = word_freq.most_common(N)

    # Print the top N most common words
    for word, freq in top_words:
        print(word, freq)

print(len(words)
#       ,'\n', words
     )

In [ ]:
# topNwords(words,100)

'''
use df[df['name'].fillna('').str.contains('heart')] to mannually check name
possible 1/0 types:

cozy,coziest vs 
spacious,large, huge,big vs
sunny, sun- , bright vs 
modern, luxury, prime,  vs
garden vs
'''
ec_dic = {'small':['cozy','coziest'],
            'big':['spacious','large', 'huge','big'],
            'light':['sunny', 'sun-', 'bright'],
            'pres':['modern', 'luxury', 'prime'],
            'garden':['garden']}

In [ ]:
def encode_name_column(df, mapping_dict):
    encoded_columns = {}
    for col_name, keywords in mapping_dict.items():
        encoded_col = df['name'].apply(lambda x: 1 if any(word in str(x).lower() for word in keywords) else 0)
        encoded_columns[col_name] = encoded_col
    encoded_df = pd.DataFrame(encoded_columns)
    return pd.concat([df, encoded_df], axis=1)

In [ ]:
df = encode_name_column(df, ec_dic)

In [ ]:
'''
convert host_name to string
guess host gender based on host_name then onehot encode it
'''

df['host_name'] = df['host_name'].astype(str)
df['host_name'].dtype
# Create instance of gender detector
d = gender.Detector()

# Define function to return gender based on name
def get_gender(name):
    gender = d.get_gender(name.title())
    if gender == 'male' or gender == 'female':
        return gender
    else:
        return 'androgynous'

# Apply the function to the 'name' column of the DataFrame and create a new 'gender' column
df['gender'] = df['host_name'].apply(get_gender)

# One-hot encode the 'gender' column
df = pd.concat([df, pd.get_dummies(df['gender'])], axis=1)

# Drop the original 'gender' column
df = df.drop('gender', axis=1)
df.shape

In [ ]:
'''
general encoding methods:
one hot can be functioned
target is tougher to generalize, wirte individually
'''
def one_hot_encode(df, feature):
    # Perform one-hot encoding
    one_hot = pd.get_dummies(df[feature], prefix=feature)

#     # drop base feature 
#     df = df.drop(feature, axis=1)
    
    # Add the encoded columns to the original DataFrame
    df = pd.concat([df, one_hot], axis=1)
    
    print(df.shape)
    print('feature_gain:', df[feature].nunique())
    
#     Return the modified DataFrame
    return df

In [ ]:
df = one_hot_encode(df, 'neighbourhood_group')

In [ ]:
df = one_hot_encode(df, 'neighbourhood')

In [ ]:
df = one_hot_encode(df, 'room_type')

In [395]:
'''
last review feature engineering
'''
df['last_review'] = pd.to_datetime(df['last_review'])
df['lr_year'] = df['last_review'].dt.year
df['lr_month'] = df['last_review'].dt.month
df['lr_day'] = df['last_review'].dt.day
df['lr_day_of_week'] = df['last_review'].dt.dayofweek
#dayoftheweek Monday=0, Sunday=6


In [302]:
df = one_hot_encode(df, 'lr_year')
df = one_hot_encode(df, 'lr_month')
df = one_hot_encode(df, 'lr_day')
df = one_hot_encode(df, 'lr_day_of_week')

(48884, 266)
feature_gain: 9
(48884, 278)
feature_gain: 12
(48884, 309)
feature_gain: 31
(48884, 316)
feature_gain: 7


In [303]:
#number of minimum nights
df['long_stay'] = df['minimum_nights'].apply(lambda x: 1 if x >50 else 0)

In [304]:
#number of revies label
df['few_reviews'] = df['number_of_reviews'].apply(lambda x: 1 if x <50 else 0)

In [305]:
#review per month label
df['few_monthly_reviews'] = df['reviews_per_month'].astype(int).apply(lambda x: 1 if x <3 else 0)

In [306]:
#calculated host listing count label
df['few_total_listing'] = df['calculated_host_listings_count'].apply(lambda x: 1 if x <20 else 0)

In [307]:
'''
availability grouped to find high variaty days
label encode high variaty days
'''
wide_variaty_availability = df.groupby('availability_365').size().sort_values(ascending=False).head(5).index
df['wide_variaty'] = df['availability_365'].apply(lambda x: 1 if x in wide_variaty_availability else 0)

In [308]:
df.shape

(48884, 321)

In [309]:
odf.columns.tolist()

['id',
 'name',
 'host_id',
 'host_name',
 'neighbourhood_group',
 'neighbourhood',
 'latitude',
 'longitude',
 'room_type',
 'price',
 'minimum_nights',
 'number_of_reviews',
 'last_review',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365']

In [310]:
notuseful = ['id',
 'name',
 'host_id',
 'host_name',
 'neighbourhood_group',
 'neighbourhood',
 'latitude',
 'longitude',
 'room_type',
 'price',
#  'minimum_nights',
#  'number_of_reviews',
 'last_review',
#  'reviews_per_month',
#  'calculated_host_listings_count',
#  'availability_365'

'lr_year',
'lr_month',
'lr_day',
'lr_day_of_week'
            ]

In [311]:
Data = df.drop(columns=notuseful)
tar = df['price']

In [406]:
Data.shape
# Data.head(3)

(48884, 306)

In [403]:
# tar

# Modeling 

In [405]:
'''
clearly regression problem
expectation is tree is better, LR is bad at capturing non linear and categorical data
expectation is boost tree should be better than RF for the outlier capture
    expectation failed becasue outlier ARE caputred but they are too scattered therefore messed up model
'''

'\nclearly regression problem\nexpectation is tree is better, LR is bad at capturing non linear and categorical data\nexpectation is boost tree should be better than RF for the outlier capture\n    expectation failed becasue outlier ARE caputred but they are too skewed therefore messed up model\n'

In [314]:
from sklearn.model_selection import train_test_split
X_trainvalid, X_test, y_trainvalid, y_test = train_test_split(Data, tar, test_size=0.15, random_state=1236)
X_train, X_valid, y_train, y_valid = train_test_split(X_trainvalid, y_trainvalid, test_size=0.175, random_state=1236)
print(y_train.mean(),y_valid.mean(),y_test.mean())
print(len(y_train),len(y_valid),len(y_test))

152.82155255404183 152.8175192519252 152.382244647484
34279 7272 7333


In [315]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score

## RF 

In [316]:
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [317]:
y_train_hat_rf = rf_model.predict(X_train)
y_valid_hat_rf = rf_model.predict(X_valid)
y_test_hat_rf = rf_model.predict(X_test)

In [318]:
print(mean_absolute_error(y_train,y_train_hat_rf))
print(mean_absolute_error(y_valid,y_valid_hat_rf))
print(mean_absolute_error(y_test,y_test_hat_rf))
print('--------------------------------')
print(explained_variance_score(y_train,y_train_hat_rf))
print(explained_variance_score(y_valid,y_valid_hat_rf))
print(explained_variance_score(y_test,y_test_hat_rf))

26.781263490328392
67.37884564958917
67.28394589858165
--------------------------------
0.8373958003463924
-0.02505485863198942
0.09098201422075514


In [404]:
# rf_model.coef_

## GB

In [319]:
from sklearn.ensemble import GradientBoostingRegressor
GB_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1)
GB_model.fit(X_train, y_train)

GradientBoostingRegressor()

In [320]:
y_train_hat_GB = GB_model.predict(X_train)
y_valid_hat_GB = GB_model.predict(X_valid)
y_test_hat_GB = GB_model.predict(X_test)

In [321]:
print(mean_absolute_error(y_train,y_train_hat_GB))
print(mean_absolute_error(y_valid,y_valid_hat_GB))
print(mean_absolute_error(y_test,y_test_hat_GB))
print('--------------------------------')
print(explained_variance_score(y_train,y_train_hat_GB))
print(explained_variance_score(y_valid,y_valid_hat_GB))
print(explained_variance_score(y_test,y_test_hat_GB))

65.48688558820223
67.13497574993593
68.1037591721927
--------------------------------
0.2933608740293152
0.08328924574671459
0.12499904660643657


## XG

In [323]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
xg_model = xgb.XGBRegressor(objective='reg:absoluteerror', n_estimators=1000, learning_rate=0.05)
xg_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             objective='reg:absoluteerror', predictor=None, ...)

In [324]:
y_train_hat_xg = xg_model.predict(X_train)
y_valid_hat_xg = xg_model.predict(X_valid)
y_test_hat_xg = xg_model.predict(X_test)

In [343]:
print(mean_absolute_error(y_train,y_train_hat_xg))
print(mean_absolute_error(y_valid,y_valid_hat_xg))
print(mean_absolute_error(y_test,y_test_hat_xg))
print('--------------------------------')
print(explained_variance_score(y_train,y_train_hat_xg))
print(explained_variance_score(y_valid,y_valid_hat_xg))
print(explained_variance_score(y_test,y_test_hat_xg))

67.38583389925893
66.70808293926964
68.15695159113068
--------------------------------
0.0637591849073027
0.04620685288203419
0.053133119773350224


##  Lasso-L1

In [326]:
from sklearn.linear_model import Lasso
L1_model = Lasso(alpha=0.1)
L1_model.fit(X_train, y_train)

Lasso(alpha=0.1)

In [327]:
y_train_hat_L1 = L1_model.predict(X_train)
y_valid_hat_L1 = L1_model.predict(X_valid)
y_test_hat_L1 = L1_model.predict(X_test)

In [328]:
print(mean_absolute_error(y_train,y_train_hat_L1))
print(mean_absolute_error(y_valid,y_valid_hat_L1))
print(mean_absolute_error(y_test,y_test_hat_L1))
print('--------------------------------')
print(explained_variance_score(y_train,y_train_hat_L1))
print(explained_variance_score(y_valid,y_valid_hat_L1))
print(explained_variance_score(y_test,y_test_hat_L1))

71.9204455787524
72.38747295992124
73.09039389151674
--------------------------------
0.1378598102588734
0.08345314489625366
0.10802905754147363


##  Ridge-L2

In [329]:
from sklearn.linear_model import Ridge
L2_model = Ridge(alpha=0.5)
L2_model.fit(X_train, y_train)

Ridge(alpha=0.5)

In [330]:
y_train_hat_L2 = L2_model.predict(X_train)
y_valid_hat_L2 = L2_model.predict(X_valid)
y_test_hat_L2 = L2_model.predict(X_test)

In [331]:
print(mean_absolute_error(y_train,y_train_hat_L2))
print(mean_absolute_error(y_valid,y_valid_hat_L2))
print(mean_absolute_error(y_test,y_test_hat_L2))
print('--------------------------------')
print(explained_variance_score(y_train,y_train_hat_L2))
print(explained_variance_score(y_valid,y_valid_hat_L2))
print(explained_variance_score(y_test,y_test_hat_L2))

72.37220508299687
73.30383094561255
73.7660012289345
--------------------------------
0.14249073977549176
0.08147215713169176
0.10765290944205241


##  LR

In [332]:
from sklearn.linear_model import LinearRegression
model_LR = LinearRegression()
model_LR.fit(X_train,y_train)

LinearRegression()

In [333]:
y_train_hat_LR = model_LR.predict(X_train)
y_valid_hat_LR = model_LR.predict(X_valid)
y_test_hat_LR = model_LR.predict(X_test)

In [334]:
print(mean_absolute_error(y_train,y_train_hat_LR))
print(mean_absolute_error(y_valid,y_valid_hat_LR))
print(mean_absolute_error(y_test,y_test_hat_LR))
print('--------------------------------')
print(explained_variance_score(y_train,y_train_hat_LR))
print(explained_variance_score(y_valid,y_valid_hat_LR))
print(explained_variance_score(y_test,y_test_hat_LR))

72.37097958172733
6826617.000477303
6907881.227889276
--------------------------------
0.1425359563233436
-1068773236009.662
-1341075099221.4766


# Improvments and follow up 

In [335]:
'''
more feautre engineering from name using NL models, also improvment on existing codes
more feature engineering from geolocation columns
proper treatment of last_review column given its 1/5 nan value but significant business impact, probally 2 project, new/return and old
figure out why XGB and GB are so bad, probabbly becasue of the outliers
'''

'\nmore feautre engineering from name using NL models, also improvment on existing codes\nmore feature engineering from geolocation columns\nproper treatment of last_review column given its 1/5 nan value but significant business impact\n'